In [1]:
import pandas as pd
import tweepy
import pymongo
import ipywidgets as wgt
from IPython.display import display
from datetime import datetime

consumer_key = 'xgih8VeTYowI3Mvi234J9A5YN'
consumer_secret = 'UJRg8LN2siNIrNS7uTMiTS4iPagV5gMQ8ePdjaT1EPVIP82Ewf'

access_token = '1043063211044601858-gdRhOxR3UaJc85GVfmYTjmk9HeDUjo'
access_token_secret = '0uN9Ej8UG43dmaqN56I8U1mrDXQ7HoMl36gpnhChLnTCu'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [2]:
#DB connection in this case Mongo DB
#col = pymongo.MongoClient()["db"]["Streaming"]
col = pymongo.MongoClient()["tweets"]["streaming"]

In [3]:
class MyStreamListener(tweepy.StreamListener):
    
    counter = 0
    
    def __init__(self, max_tweets=1000, *args, **kwargs):
        self.max_tweets = max_tweets
        self.counter = 0
        super().__init__(*args, **kwargs)
    
    def on_connect(self):
        self.counter = 0
        self.start_time = datetime.now()
    
    def on_status(self, status):
        # Increment counter
        self.counter += 1
        
        # Store tweet to MongoDB
        col.insert_one(status._json)
        
        
        if self.counter % 1 == 0:
            value = int(100.00 * self.counter / self.max_tweets)
            mining_time = datetime.now() - self.start_time
            progress_bar.value = value
            html_value = """<span class="label label-primary">Tweets/Sec: %.1f</span>""" % (self.counter / max([1,mining_time.seconds]))
            html_value += """ <span class="label label-success">Progress: %.1f%%</span>""" % (self.counter / self.max_tweets * 100.0)
            html_value += """ <span class="label label-info">ETA: %.1f Sec</span>""" % ((self.max_tweets - self.counter) / (self.counter / max([1,mining_time.seconds])))
            wgt_status.value = html_value
            #print("%s/%s" % (self.counter, self.max_tweets))
            if self.counter >= self.max_tweets:
                myStream.disconnect()
                print("Finished")
                print("Total Mining Time: %s" % (mining_time))
                print("Tweets/Sec: %.1f" % (self.max_tweets / mining_time.seconds))
                progress_bar.value = 0
                
    
myStreamListener = MyStreamListener(max_tweets=10)
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [4]:
#Connecting
keywords = ["Donald Trump",
           "DonaldTrump"]

#Visualize
progress_bar = wgt.IntProgress(value=0)
display(progress_bar)
wgt_status = wgt.HTML(value="""<span class="label label-primary">Tweets/Sec: 0.0</span>""")
display(wgt_status)

#Start filter
for error_counter in range(20):
    try:
        myStream.filter(track=keywords)
        print("Tweets collected: %s" % col.count())
        break
    except:
        print("ERROR# %S" % (error_counter + 1))

IntProgress(value=0)

HTML(value='<span class="label label-primary">Tweets/Sec: 0.0</span>')

Finished
Total Mining Time: 0:00:03.173645
Tweets/Sec: 3.3
Tweets collected: 10


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  from ipykernel import kernelapp as app


In [5]:
#Look at one of the tweets collected
col.find_one()

{'_id': ObjectId('5bb35fef2ac6b32dc51d0e94'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Oct 02 12:09:20 +0000 2018',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'politicususa.com/2018/10/01/kav…',
    'expanded_url': 'https://www.politicususa.com/2018/10/01/kavanaugh-trump-term-limits.html',
    'indices': [110, 133],
    'url': 'https://t.co/KPtfKbF7O7'}],
  'user_mentions': [{'id': 14792049,
    'id_str': '14792049',
    'indices': [3, 16],
    'name': 'PoliticusUSA',
    'screen_name': 'politicususa'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 1047096189714386947,
 'id_str': '1047096189714386947',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'possibly_sensitive': False,
 'quote_count': 0,
 'reply_count': 0,

In [6]:
#DataFrame to visualize the data
dataset = [{"created_at": item["created_at"],
            "id": item["id"],
           # "favourites_count": item["favourites_count"],
            "text": item["text"],
            "user": "@%s" % item["user"]["screen_name"],
            "source": item["source"],
           } for item in col.find()]

dataset = pd.DataFrame(dataset)
dataset

,created_at,id,source,text,user
0,Tue Oct 02 12:09:20 +0000 2018,1047096189714386947,"<a href=""http://twitter.com/#!/download/ipad"" ...",RT @politicususa: A Supreme Court Justice Bret...,@malama00
1,Tue Oct 02 12:09:20 +0000 2018,1047096190821519363,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","#Video: Sebut RI Sahabat, @realDonaldTrump Sam...",@cnbcindonesia
2,Tue Oct 02 12:09:20 +0000 2018,1047096191882805248,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","@larepublica_uy para mi el mejor ""ministro"" es...",@AbrahamNunez_uy
3,Tue Oct 02 12:09:20 +0000 2018,1047096191794761728,"<a href=""http://twitter.com/download/android"" ...",RT @matthewamiller: Every time I think Donald ...,@Lloydie_D
4,Tue Oct 02 12:09:21 +0000 2018,1047096195120869377,"<a href=""http://twitter.com/download/iphone"" r...",RT @MichaelAvenatti: Donald Trump and his righ...,@artfoundryman
5,Tue Oct 02 12:09:21 +0000 2018,1047096196349747201,"<a href=""http://twitter.com/download/iphone"" r...",RT @MichaelAvenatti: Donald Trump and his righ...,@thereweys
6,Tue Oct 02 12:09:22 +0000 2018,1047096197025017857,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @MichaelAvenatti: Donald Trump and his righ...,@PdToLoseWeight
7,Tue Oct 02 12:09:22 +0000 2018,1047096197993787392,"<a href=""http://www.example.com"" rel=""nofollow...",Donald Trump calls Hillary Clinton the biggest...,@denisekushler24
8,Tue Oct 02 12:09:22 +0000 2018,1047096198350352384,"<a href=""http://twitter.com/download/android"" ...",@WhiteHouse @realDonaldTrump Donald Trump,@Name34252905
9,Tue Oct 02 12:09:23 +0000 2018,1047096202016247810,"<a href=""http://twitter.com/download/iphone"" r...",RT @MichaelAvenatti: Donald Trump and his righ...,@ObtuseToQuit


In [7]:
dataset.to_csv("data.csv", encoding='utf-8')